In [1]:
OUTPUT_PATH = '../../data/'

In [2]:
%run '../../schema_definition/schema.py'

In [3]:
import sys
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [4]:
from pyspark.shell import spark
from pyspark.sql import functions as F

21/10/16 04:14:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.8.9 (default, Aug 21 2021 15:53:23)
Spark context Web UI available at http://localhost:4042
Spark context available as 'sc' (master = local[*], app id = local-1634318074587).
SparkSession available as 'spark'.


## List post codes based on fastest response
***
- Fastest response hint ( Refer columns Request date and implementation Date )
- When the number of events reach 1000, output the events to a JSON file.
- The output filenames should have a batch number e.g. the second 1000 records will go into a file called 2.json and so on.

In [34]:
gold_df = spark.read.parquet(OUTPUT_PATH + "gold")
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT DISTINCT
                        PostCode, Response
                        FROM fastest_response
                        ORDER BY Response DESC
                        """
fastest_response_df = spark.sql(fastest_response_query)
drop_date_column  = fastest_response_df.drop('date')
drop_date_column.coalesce(1).write.option("maxRecordsPerFile", 1000).json(OUTPUT_PATH + 'response', mode='overwrite')
df = drop_date_column.toPandas()
df

,PostCode,Response
0,2081,-1
1,2586,-1
2,2582,-1
3,2722,-1
4,2587,-1
...,...,...
147,2582,-368
148,2582,-378
149,2582,-424
150,2582,-1020


In [40]:
import plotly.express as px

fig = px.scatter(df, x='Response', y='PostCode', title='Fastest response')
fig.show()